# AWS Athena: Rows and Columns

## Experiment:

- Better understand working with "serverless databases"
- Get some actual data on row-based vs column-based storage

Data source: [USAspending.gov download center](https://www.usaspending.gov/#/download_center/custom_award_data)

- detailed information about U.S. federal grants, direct payments, loans, and insurance "obligations"
- U.S. fiscal years 2001 - 2018 (partial)
- All federal agencies
- ~40.5 million rows
- row-based format = uncompressed .csv (19.5 GB)
- column-based format = parquet (3 GB)

![media/usaspending.png](media/usaspending.png)

## Step 1: Put Structured Data on S3

Download .csv files about specific grants, direct payments, loans, and insurance "obligations" made by the U.S. Federal Government, unzip the files, and upload them to S3. 

Source: [USAspending.gov download center](https://www.usaspending.gov/#/download_center/custom_award_data)

For this demo:

- all federal agencies
- U.S. fiscal years 2001 - 2018 (partial)
- about 19 GB

In [ ]:
!aws s3 ls s3://usaspending-files/awards/assistance/ --summarize --recursive --human-readable

## Step 2: Update the Metastore

Describe the layout of the .csv Athena

```sql
CREATE EXTERNAL TABLE IF NOT EXISTS usaspending.assistance (
  `award_id_fain` string,
  `modification_number` string,
  `award_id_uri` string,
  `sai_number` string,
  `federal_action_obligation` double,
  `non_federal_funding_amount` double,
  `total_funding_amount` double,
  `face_value_of_loan` double,
  `original_subsidy_cost` double,
  `total_subsidy_cost` double,
  `total_loan_value` double,
  `action_date` date,
  `period_of_performance_start_date` date,
  `period_of_performance_current_end_date` date,
  `awarding_agency_code` string,
  `awarding_agency_name` string,
  `awarding_sub_agency_code` string,
  `awarding_sub_agency_name` string,
  `awarding_office_code` string,
  `awarding_office_name` string,
  `funding_agency_code` string,
  `funding_agency_name` string,
  `funding_sub_agency_code` string,
  `funding_sub_agency_name` string,
  `funding_office_code` string,
  `funding_office_name` string,
  `recipient_duns` string,
  `recipient_name` string,
  `recipient_parent_name` string,
  `recipient_parent_duns` string,
  `recipient_country_code` string,
  `recipient_country_name` string,
  `recipient_address_line_1` string,
  `recipient_address_line_2` string,
  `recipient_city_code` string,
  `recipient_city_name` string,
  `recipient_county_code` string,
  `recipient_county_name` string,
  `recipient_state_code` string,
  `recipient_state_name` string,
  `recipient_zip_code` string,
  `recipient_zip_last_4_code` string,
  `recipient_congressional_district` string,
  `recipient_foreign_city_name` string,
  `recipient_foreign_province_name` string,
  `recipient_foreign_postal_code` string,
  `primary_place_of_performance_country_code` string,
  `primary_place_of_performance_country_name` string,
  `primary_place_of_performance_code` string,
  `primary_place_of_performance_city_name` string,
  `primary_place_of_performance_county_code` string,
  `primary_place_of_performance_county_name` string,
  `primary_place_of_performance_state_name` string,
  `primary_place_of_performance_zip_4` string,
  `primary_place_of_performance_congressional_district` string,
  `primary_place_of_performance_foreign_location` string,
  `cfda_number` string,
  `cfda_title` string,
  `assistance_type_code` string,
  `assistance_type_description` string,
  `award_description` string,
  `business_funds_indicator_code` string,
  `business_funds_indicator_description` string,
  `business_types_code` string,
  `business_types_description` string,
  `correction_delete_indicator_code` string,
  `correction_delete_indicator_description` string,
  `action_type_code` string,
  `action_type_description` string,
  `record_type_code` string,
  `record_type_description` string,
  `last_modified_date` timestamp 
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
) LOCATION 's3://usaspending-files/awards/assistance/'
TBLPROPERTIES ('has_encrypted_data'='false', 'skip.header.line.count'='1');
```

## Step 3: Create a Parquet-Based Version of the Table

[Apache Parquet](https://parquet.apache.org/) is a columnar storage format.

To see how switching to a column-based format can impact amount of data read, create a second Athena table. Same data, but this one will sit on top of a Parquet version of the data.

(note: h/t [this post](https://medium.com/searce/convert-csv-json-files-to-apache-parquet-using-aws-glue-a760d177b45f) on using AWS Glue to convert .csv data to Parquet.

In [ ]:
!aws s3 ls s3://usaspending-files/awards-parquet/assistance/ --summarize --recursive --human-readable

## Demo Query
Get some information about U.S. Federal grants

Total award amounts summarized by:

- year
- name of award recipient
- program title


```sql
SELECT
  date_format(date_parse(action_date, '%Y-%m-%d'), '%Y') as year,
  -- date_format(action_date, '%Y') as year,
  recipient_name as recipient,
  cfda_title as program,
  SUM(federal_action_obligation) AS total
FROM
  usaspending.assistance_parquet
WHERE (
  recipient_name IN ('SETI INSTITUTE', 'UNIVERSITY OF MASSACHUSETTS')
  OR recipient_city_name IN ('AMHERST', 'NORTHAMPTON'))
GROUP BY 1, 2, 3
ORDER BY 1, 2, 3
```

## Results:

* Table based on .csv: scanned 19.52 GB
* Tasble based on parquet: scanned 469 MB

## Thoughts and Caveats

* This wasn't a totally fair comparison b/c the .csvs weren't compressed or partitioned
* This type of experiment doesn't reflect production-level considerations:  
    * less flexible updates
    * schema-on-write vs schema-on-read
    * compacting partitions
    * separation storage and compute = more moving parts

As always, before adopting, would be important to understand what you're solving for and consider the trade-offs. 


## Connect to Athena via Python

In [7]:
from pyathenajdbc import connect

conn = connect(
    s3_staging_dir='s3://aws-athena-query-results-521138137878-us-east-2/',
    region_name='us-east-2'
)

try:
    with conn.cursor() as cursor:
        cursor.execute("""SELECT * FROM usaspending.assistance LIMIT 10""")
        print(cursor.description)
        print(cursor.fetchall())
finally:
    conn.close()

[('award_id_fain', 'VARCHAR', 255, None, 255, 0, 1), ('modification_number', 'VARCHAR', 255, None, 255, 0, 1), ('award_id_uri', 'VARCHAR', 255, None, 255, 0, 1), ('sai_number', 'VARCHAR', 255, None, 255, 0, 1), ('federal_action_obligation', 'DOUBLE', 24, None, 53, 0, 1), ('non_federal_funding_amount', 'DOUBLE', 24, None, 53, 0, 1), ('total_funding_amount', 'DOUBLE', 24, None, 53, 0, 1), ('face_value_of_loan', 'DOUBLE', 24, None, 53, 0, 1), ('original_subsidy_cost', 'DOUBLE', 24, None, 53, 0, 1), ('total_subsidy_cost', 'DOUBLE', 24, None, 53, 0, 1), ('total_loan_value', 'DOUBLE', 24, None, 53, 0, 1), ('action_date', 'DATE', 10, None, 10, 0, 1), ('period_of_performance_start_date', 'DATE', 10, None, 10, 0, 1), ('period_of_performance_current_end_date', 'DATE', 10, None, 10, 0, 1), ('awarding_agency_code', 'VARCHAR', 255, None, 255, 0, 1), ('awarding_agency_name', 'VARCHAR', 255, None, 255, 0, 1), ('awarding_sub_agency_code', 'VARCHAR', 255, None, 255, 0, 1), ('awarding_sub_agency_name', 

## Athena Results to Pandas DataFrame

In [12]:
import contextlib
from pyathenajdbc import connect
from pyathenajdbc.util import as_pandas

with contextlib.closing(
    connect(s3_staging_dir='s3://aws-athena-query-results-521138137878-us-east-2/',
        region_name='us-east-2')) as conn:
    with conn.cursor() as cursor:
        cursor.execute("""
        SELECT * FROM usaspending.assistance LIMIT 10
        """)
        df = as_pandas(cursor)
print(df.describe())


       federal_action_obligation  non_federal_funding_amount  \
count               1.000000e+01                   10.000000   
mean                6.821878e+05                15737.900000   
std                 1.736623e+06                49767.609588   
min                -1.923400e+04                    0.000000   
25%                 2.029250e+03                    0.000000   
50%                 2.456200e+04                    0.000000   
75%                 4.051562e+05                    0.000000   
max                 5.585358e+06               157379.000000   

       total_funding_amount  face_value_of_loan  original_subsidy_cost  \
count          1.000000e+01        1.000000e+01                    2.0   
mean           6.979257e+05        1.901000e+05                    0.0   
std            1.735501e+06        5.837923e+05                    0.0   
min           -1.923400e+04        0.000000e+00                    0.0   
25%            2.029250e+03        0.000000e+00      